In [2]:
from DbConnector import DbConnector
#from part2 import Database
from tabulate import tabulate

# Set up the program
from dotenv import load_dotenv
import os
import numpy as np
from datetime import datetime, timedelta
from haversine import haversine, Unit
from pprint import pprint
import pymongo

load_dotenv()

connector = DbConnector()

You are connected to the database: assignment3
-----------------------------------------------



### Task 1

How many users, activities and trackpoints are there in the dataset (after it is inserted into the database)

### Task 2
Find the average, maximum and minimum number of trackpoints per user

##### Average

##### Maximum

##### Minimum

### Task 3

Find the top 15 users with the highest number of activities.

In [6]:
collection = connector.db["Activity"]

rows = collection.aggregate(
    [
        {"$group": {"_id":  "$user_id", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": 15},
    ]
)

for row in rows:
    pprint(row)

{'_id': {'user_id': '025'}, 'count': 715}
{'_id': {'user_id': '128'}, 'count': 519}
{'_id': {'user_id': '062'}, 'count': 406}
{'_id': {'user_id': '041'}, 'count': 399}
{'_id': {'user_id': '004'}, 'count': 346}
{'_id': {'user_id': '140'}, 'count': 345}
{'_id': {'user_id': '017'}, 'count': 265}
{'_id': {'user_id': '003'}, 'count': 261}
{'_id': {'user_id': '014'}, 'count': 236}
{'_id': {'user_id': '030'}, 'count': 210}
{'_id': {'user_id': '011'}, 'count': 201}
{'_id': {'user_id': '039'}, 'count': 198}
{'_id': {'user_id': '034'}, 'count': 180}
{'_id': {'user_id': '000'}, 'count': 155}
{'_id': {'user_id': '002'}, 'count': 146}


### Task 4
Find all users who have taken a bus.

### Task 5
List the top 10 users by their amount of different transportation modes.

### Task 6
Find activities that are registered multiple times. You should find the query even
if it gives zero result.

Assumption: Registered multiple times, meaning duplicates with the same user_id, transportation_mode, start_date_time, end_date_time.

In [13]:
collection = connector.db["Activity"]

rows = collection.aggregate(
    [
        {
            "$lookup": {
                "from": "Activity",
                "let": {
                    "a_id": "$_id",
                    "a_user_id": "$user_id",
                    "a_transportation_mode": "$transportation_mode",
                    "a_start_date_time": "$start_date_time",
                    "a_end_date_time": "$end_date_time",
                },
                "pipeline": [
                    {
                        "$match": {
                            "$and": [
                                {"a_id": {
                                    "$ne":  "$_id"
                                }},
                                {"a_user_id": {
                                    "$eq": "$user_id"
                                }},
                                {
                                    "a_transportation_mode": {
                                        "$eq": "$transportation_mode"
                                    }
                                },
                                {
                                    "a_start_date_time": {
                                        "$eq":"$start_date_time"
                                    }

                                },
                                {
                                    "a_end_date_time": {
                                        "$eq":"$end_date_time"
                                    }

                                },
                            ]
                        }
                    }
                ],
                "as": "duplicates"
            }
        },
        {"$match": {
            "duplicates": {
                "$ne": []
            }
        }}
    ]
)

for row in rows:
    print(row)

{'_id': 1, 'user_id': '135', 'transportation_mode': None, 'start_time': '2009-01-03 01:21:34', 'end_time': '2009-01-03 05:40:31', 'duplicates': []}
{'_id': 2, 'user_id': '135', 'transportation_mode': None, 'start_time': '2009-01-02 04:31:27', 'end_time': '2009-01-02 04:41:05', 'duplicates': []}
{'_id': 3, 'user_id': '135', 'transportation_mode': None, 'start_time': '2009-01-27 03:00:04', 'end_time': '2009-01-27 04:50:32', 'duplicates': []}
{'_id': 4, 'user_id': '135', 'transportation_mode': None, 'start_time': '2009-01-10 01:19:47', 'end_time': '2009-01-10 04:42:47', 'duplicates': []}
{'_id': 5, 'user_id': '135', 'transportation_mode': None, 'start_time': '2009-01-14 12:17:57', 'end_time': '2009-01-14 12:30:53', 'duplicates': []}
{'_id': 6, 'user_id': '135', 'transportation_mode': None, 'start_time': '2009-01-12 01:41:22', 'end_time': '2009-01-12 02:14:01', 'duplicates': []}
{'_id': 7, 'user_id': '135', 'transportation_mode': None, 'start_time': '2008-12-24 14:42:07', 'end_time': '2008

### Task 7
a) Find the number of users that have started an activity in one day and ended the activity the next day.

b) List the transportation mode, user id and duration for these activities.

Assumption: As the question explicitly mentions transportation mode we do not count the rows in the Activity table with transportation mode equals NULL as relevant to this question. Thus, there are less rows in this sub task. However, if it is the case to include these NULL rows then we only need to remove the 'AND transportation_mode IS NOT NULL' part.

### Task 8

Find the number of users which have been close to each other in time and space. Close is defined as the same space (50 meters) and for the same half minute (30 seconds)

### Task 9
Find the top 15 users who have gained the most altitude meters.

In [ ]:
collection = connector.db['Activity']

rows = collection.aggregate([
    {"$lookup": {
        "from": "TrackPoint",
        "let": {
            "altitude": "$altitude"
        }
    }}
])

### Task 10
Find the users that have traveled the longest total distance in one day for each
transportation mode.

### Task 11
Find all users who have invalid activities, and the number of invalid activities per user.

### Task 12
Find all the users who have registered transportation_mode and their most used transportation_mode

In [18]:
collection = connector.db["Activity"]

rows = collection.aggregate(
    [
        {"$match": {"transportation_mode": {"$ne": None}}},
        {
            "$group": {
                "_id": {
                    "user_id": "$user_id",
                    "transportation_mode": "$transportation_mode",
                },
                "count": {"$sum": 1},
            }
        },
        {
            "$lookup": {
                "from": "User",
                "let": {
                    "a_id": "$_id",
                    "a_has_labels": "$has_labels",
                },
                "pipeline": [{"$match": {"$expr": {"$eq": ["$$a_id", "$user_id"]}}}],
                "as": "saves_transportation",
            }
        },
        {"$match": {"saves_transportation": {"$ne": {"$a_has_labels": False}}}},
        {"$sort": {"_id.user_id": 1, "count": -1}},
        {
            "$group": {
                "_id": "$_id.user_id",
                "most_used_transportation_mode": {"$first": "$_id.transportation_mode"},
            }
        },
        {"$sort": {"_id.user_id": 1}},
    ]
)

for row in rows:
    pprint(row)

{'_id': '117', 'most_used_transportation_mode': 'walk'}
{'_id': '020', 'most_used_transportation_mode': 'bike'}
{'_id': '065', 'most_used_transportation_mode': 'bike'}
{'_id': '097', 'most_used_transportation_mode': 'bike'}
{'_id': '062', 'most_used_transportation_mode': 'bus'}
{'_id': '107', 'most_used_transportation_mode': 'walk'}
{'_id': '161', 'most_used_transportation_mode': 'walk'}
{'_id': '167', 'most_used_transportation_mode': 'bike'}
{'_id': '021', 'most_used_transportation_mode': 'walk'}
{'_id': '125', 'most_used_transportation_mode': 'bike'}
{'_id': '058', 'most_used_transportation_mode': 'walk'}
{'_id': '128', 'most_used_transportation_mode': 'car'}
{'_id': '139', 'most_used_transportation_mode': 'bike'}
{'_id': '089', 'most_used_transportation_mode': 'car'}
{'_id': '052', 'most_used_transportation_mode': 'bus'}
{'_id': '092', 'most_used_transportation_mode': 'walk'}
{'_id': '115', 'most_used_transportation_mode': 'car'}
{'_id': '163', 'most_used_transportation_mode': 'bike